# Improved simple CNN

In [9]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import scipy.misc as smp
import matplotlib.pyplot as plt
import numpy as np
import os, shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, confusion_matrix
from keras import models, optimizers, Sequential, regularizers, layers
from keras.models import load_model, model_from_json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import to_categorical
from keras.applications import VGG16, VGG19
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers import Flatten, Dropout, Reshape, Dense, GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
import itertools

np.random.seed(123)

In [2]:
directory = 'data/train_images/'

In [3]:
data_all = ImageDataGenerator(rescale=1./255).flow_from_directory( 
        directory, 
        target_size=(224, 224), 
        batch_size = 3662, 
        seed = 123)

Found 3662 images belonging to 5 classes.


### Running with the same (bad) data setup as the baseline cnn for comparison

In [4]:
images, labels = next(data_all)

In [5]:
X_model, X_test, y_model, y_test = train_test_split(images, labels, test_size=0.20, random_state=123, stratify = labels)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X_model, y_model, test_size=0.20, random_state=123, stratify = y_model)

### The baseline CNN structure can be seen in the cell below

In [ ]:
# baseline = models.Sequential()
# baseline.add(layers.Conv2D(224, (3, 3), activation='relu', input_shape=(224,224,3)))
# baseline.add(layers.MaxPooling2D((2, 2)))
# baseline.add(layers.Conv2D(112, (3, 3), activation='relu'))
# baseline.add(layers.MaxPooling2D((2, 2)))
# baseline.add(layers.Flatten())
# baseline.add(layers.Dropout(0.5))
# baseline.add(layers.Dense(56, activation='relu'))
# baseline.add(layers.Dense(5, activation='softmax'))

# baseline.compile(loss='categorical_crossentropy',
#               optimizer="sgd",
#               metrics=['acc'])

In [7]:
cnn = models.Sequential()
cnn.add(layers.Conv2D(224, (3, 3), activation='relu', input_shape=(224,224,3)))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Conv2D(112, (3, 3), activation='relu'))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Conv2D(56, (3, 3), activation='relu'))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Flatten())
cnn.add(layers.Dense(300, activation='relu'))
cnn.add(layers.Dropout(0.3))
cnn.add(layers.Dense(150, activation='relu'))
cnn.add(layers.Dense(5, activation='softmax'))

cnn.compile(loss='categorical_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
checkpoint = ModelCheckpoint("cnn_increased_complexity.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [11]:
len(X_train)

2343

In [13]:
cnn_hist = cnn.fit(X_train,
                y_train,
                epochs=50,
                batch_size=64,
                validation_data=(X_val, y_val),
                callbacks=[early, checkpoint])

Instructions for updating:
Use tf.cast instead.
Train on 2343 samples, validate on 586 samples
Epoch 1/50
2343/2343 [==============================] - 641s 274ms/step - loss: 1.3430 - acc: 0.4836 - val_loss: 1.1938 - val_acc: 0.5956

Epoch 00001: val_acc improved from -inf to 0.59556, saving model to cnn_increased_complexity.h5
Epoch 2/50
2343/2343 [==============================] - 637s 272ms/step - loss: 1.1706 - acc: 0.5412 - val_loss: 1.1455 - val_acc: 0.4932

Epoch 00002: val_acc did not improve from 0.59556
Epoch 3/50
2343/2343 [==============================] - 634s 270ms/step - loss: 1.0681 - acc: 0.5984 - val_loss: 0.9558 - val_acc: 0.6621

Epoch 00003: val_acc improved from 0.59556 to 0.66212, saving model to cnn_increased_complexity.h5
Epoch 4/50
2343/2343 [==============================] - 633s 270ms/step - loss: 0.9037 - acc: 0.6756 - val_loss: 0.9156 - val_acc: 0.6775

Epoch 00007: val_acc did not improve from 0.68942
Epoch 8/50
2343/2343 [==============================] 

2343/2343 [==============================] - 634s 271ms/step - loss: 0.7223 - acc: 0.7328 - val_loss: 0.7613 - val_acc: 0.7235

Epoch 00043: val_acc did not improve from 0.74232
Epoch 44/50
2343/2343 [==============================] - 634s 271ms/step - loss: 0.7187 - acc: 0.7332 - val_loss: 0.7395 - val_acc: 0.7338

Epoch 00044: val_acc did not improve from 0.74232
Epoch 45/50
2343/2343 [==============================] - 636s 271ms/step - loss: 0.7252 - acc: 0.7354 - val_loss: 0.7365 - val_acc: 0.7491

Epoch 00045: val_acc improved from 0.74232 to 0.74915, saving model to cnn_increased_complexity.h5
Epoch 46/50
2343/2343 [==============================] - 634s 270ms/step - loss: 0.7153 - acc: 0.7350 - val_loss: 0.7550 - val_acc: 0.7270

Epoch 00046: val_acc did not improve from 0.74915
Epoch 47/50
2343/2343 [==============================] - 633s 270ms/step - loss: 0.7103 - acc: 0.7362 - val_loss: 0.7622 - val_acc: 0.7457

Epoch 00047: val_acc did not improve from 0.74915
Epoch 48/50
2

In [ ]:
2+2